In [2]:
import pandas as pd

# Замените на свой файл
df = pd.read_csv('2_1.csv')  # 👈 сюда подставь имя своего .csv файла
df.head()

,Rank,Peak,All Time Peak,Actual gross,Adjusted gross (in 2022 dollars),Artist,Tour title,Year(s),Shows,Average gross,Ref.
0,1,1,2,"$780,000,000","$780,000,000",Taylor Swift,The Eras Tour †,2023–2024,56,"$13,928,571",[1]
1,2,1,7[2],"$579,800,000","$579,800,000",Beyoncé,Renaissance World Tour,2023,56,"$10,353,571",[3]
2,3,1[4],2[5],"$411,000,000","$560,622,615",Madonna,Sticky & Sweet Tour ‡[4][a],2008–2009,85,"$4,835,294",[6]
3,4,2[7],10[7],"$397,300,000","$454,751,555",Pink,Beautiful Trauma World Tour,2018–2019,156,"$2,546,795",[7]
4,5,2[4],NaN,"$345,675,146","$402,844,849",Taylor Swift,Reputation Stadium Tour,2018,53,"$6,522,173",[8]


In [3]:
# Общая информация
print("🔍 Информация о данных:")
print(df.info())

# Основные статистики
print("\n📊 Описательные статистики:")
print(df.describe(include='all'))

# Проверка на пропуски
print("\n🧹 Пропущенные значения:")
print(df.isnull().sum())



🔍 Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Rank                              20 non-null     int64 
 1   Peak                              9 non-null      object
 2   All Time Peak                     6 non-null      object
 3   Actual gross                      20 non-null     object
 4   Adjusted gross (in 2022 dollars)  20 non-null     object
 5   Artist                            20 non-null     object
 6   Tour title                        20 non-null     object
 7   Year(s)                           20 non-null     object
 8   Shows                             20 non-null     int64 
 9   Average gross                     20 non-null     object
 10  Ref.                              20 non-null     object
dtypes: int64(2), object(9)
memory usage: 1.8+ KB
None

📊 Описательн

In [6]:
df.columns = df.columns.str.strip()

# Очистка денежных столбцов от $ и запятых, приведение к float
money_cols = ['Actual gross', 'Adjusted gross (in 2022 dollars)', 'Average gross']

for col in money_cols:
    df[col] = df[col].str.replace(r'[\$,]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Проверим пропуски после преобразования
print(df[money_cols].isnull().sum())

# Можно заполнить пропуски средним или медианой, если нужно
# Например:
# df['Actual gross'].fillna(df['Actual gross'].median(), inplace=True)

# Посмотрим на итоговый датасет
print(df.info())
print(df.head())


KeyError: 'Actual gross'

In [ ]:
df.columns = df.columns.str.strip()  # Названия колонок
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_clean = df.dropna()


In [ ]:
df_clean['Salary'] = df_clean['Salary'].replace(r'[^0-9\-\–]', '', regex=True)


In [ ]:
def salary_range_quality(df, salary_col='Salary'):
    pattern = re.compile(r'^\d+\s*[-–]\s*\d+$')
    valid = df[salary_col].astype(str).apply(lambda x: bool(pattern.match(x)))
    return valid.mean()

print("Метрика качества диапазона зарплат:", salary_range_quality(df_clean))


In [ ]:
Задание 2

In [ ]:
# Замените путь на свой .csv файл
df = pd.read_csv('<Название файла>')
df.columns = df.columns.str.strip()  # Названия колонок
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df_clean = df.dropna()
df_clean = df_clean.drop_duplicates()

In [ ]:
def parse_salary_range(s):
    try:
        parts = re.findall(r'\d+', str(s))
        if len(parts) >= 2:
            low, high = map(int, parts[:2])
            return low, high, (low + high) / 2
        elif len(parts) == 1:
            return int(parts[0]), int(parts[0]), int(parts[0])
    except:
        return np.nan, np.nan, np.nan
    return np.nan, np.nan, np.nan

salary_data = df_clean['Salary'].apply(parse_salary_range)
df_clean[['Salary_Min', 'Salary_Max', 'Salary_Mid']] = pd.DataFrame(salary_data.tolist(), index=df_clean.index)


In [ ]:
def summary_stats(series, name=''):
    print(f"Статистика для {name}:")
    print("Среднее:", series.mean())
    print("Медиана:", series.median())
    print("Дисперсия:", series.var())

    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    filtered = series[(series >= q1 - 1.5 * iqr) & (series <= q3 + 1.5 * iqr)]

    print("Среднее без выбросов:", filtered.mean())
    print("Медиана без выбросов:", filtered.median())
    print("Дисперсия без выбросов:", filtered.var())
    print()

summary_stats(df_clean['Salary_Min'], 'Минимум зарплаты')
summary_stats(df_clean['Salary_Max'], 'Максимум зарплаты')
summary_stats(df_clean['Salary_Mid'], 'Средняя зарплата')


In [ ]:
df_clean['Salary_Final'] = df_clean['Salary_Mid']
